In [1]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests

## SCRAPE PRODUCTS

In [2]:
def get_links(table_tag):
    links = []

    for row_data in table_tag.select("tr"):
        link_tag = row_data.find('a', href=True)

        if(link_tag == None):
            continue

        links.append('https://www.mapa.gob.es/app/consultafertilizante/' + link_tag.get('href'))  

    return links  

In [3]:
def get_products(url):
    page = requests.get(url)
    page.encoding='utf-8'
    pagetext = page.text
    tree = BeautifulSoup(pagetext)

    table_tag = tree.select("table")[1]
    links = get_links(table_tag)
    tab_data = [[item.text for item in row_data.select("th,td")]
                for row_data in table_tag.select("tr")]

    data = tab_data[2::]

    for i in range(0,len(data)):
        data[i].append(links[i])

    return data


In [6]:
base_url = 'https://www.mapa.gob.es/app/consultafertilizante/ListadoFertilizantes.aspx?Page='
products = []
n_pages = 70

for i in range(0,n_pages + 1):
    partial_products = get_products(base_url+str(i))
    for p in partial_products:
        products.append(p)

In [7]:
len(products)

1412

In [8]:
products[0]

['F0000063/2021',
 'Abono órgano-mineral NPK',
 'BOY 12',
 'JOSE FUSTE, S.A.',
 '27/04/2011',
 'https://www.mapa.gob.es/app/consultafertilizante/DetalleFertilizante.aspx?clave=63']

## SCRAPE COMPOSITION

In [9]:
components = ['Nitrógeno(N) Total', 'Anhidrido fosfórico (P2O5)', 'Óxido de potasio (K2O)', 'Boro (B)', 'Cobalto (Co)', 'Cobre (Cu)', 'Hierro (Fe)','Manganeso (Mn)', 'Molibdeno (Mb)','Cinc (Zn)']
equivalencies = {
    'Nitrógeno(N) Total': 'Nitrógeno (N)', 
    'Anhidrido fosfórico (P2O5)' : 'Fósforo (P)', 
    'Óxido de potasio (K2O)': 'Potasio (K)'
}

In [10]:
def get_composition(url):
    global components
    global equivalencies
    page = requests.get(url)
    page.encoding='utf-8'
    pagetext = page.text

    tree = BeautifulSoup(pagetext)
    if(len(tree.select("table"))>2):
        table_tag = tree.select("table")[3]
        tab_data = [[item.text for item in row_data.select("th,td")]
                        for row_data in table_tag.select("tr")]

        composition = ''
        for e in tab_data:
            if(e[0] in components and e[1] != '-'):
                if e[0] in equivalencies.keys():
                    composition += e[1] +'% ' + equivalencies[e[0]] + '; '
                else: 
                    composition += e[1] +'% ' + e[0] + '; '

        return composition[0:-2]
    else:
        return None

In [11]:
products_df = pd.DataFrame(products)
products_df['Composicion'] = products_df.apply(lambda row: get_composition(row[5]), axis=1)
products_df.columns= ['Código', 'Tipo', 'Nombre comercial', 'Fabricante', 'F. de registro', 'Url', 'Composicion']

In [12]:
products_df.head()

,Código,Tipo,Nombre comercial,Fabricante,F. de registro,Url,Composicion
0,F0000063/2021,Abono órgano-mineral NPK,BOY 12,"JOSE FUSTE, S.A.",27/04/2011,https://www.mapa.gob.es/app/consultafertilizan...,"6,0% Nitrógeno (N); 10,0% Fósforo (P); 6,0% Po..."
1,F0000065/2021,Abono órgano-mineral NPK,BOY 10,"JOSE FUSTE, S.A.",27/04/2011,https://www.mapa.gob.es/app/consultafertilizan...,"5,0% Nitrógeno (N); 7,0% Fósforo (P); 10,0% Po..."
2,F0000347/2028,Abono orgánico NPK de origen animal,HORTYFLOR,"ESTEVENATUR, S.L.",09/04/2018,https://www.mapa.gob.es/app/consultafertilizan...,"3,0% Nitrógeno (N); 3,0% Fósforo (P); 3,0% Pot..."
3,F0000791/2028,Enmienda orgánica húmica,NATURCOMPLET,DAYMSA,27/03/2018,https://www.mapa.gob.es/app/consultafertilizan...,"1,0% Hierro (Fe)"
4,F0001455/2020,Enmienda orgánica húmica,CULTIVIT Fe,"AGRIBECO, S.L.",07/07/2010,https://www.mapa.gob.es/app/consultafertilizan...,"2,3% Nitrógeno (N); 1,8% Fósforo (P); 1,5% Pot..."
